<a href="https://colab.research.google.com/github/aithaprasad/related-languages/blob/main/Uday_NLP_5_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def get_sentences_from_file(file):
  sentences = []
  temp = []
  data = open(file, 'r')
  for line in data:
    token = line.rstrip("\n")
    if token == '<s>': temp = []
    elif token == '</s>': sentences.append(temp)
    else: temp.append(line.split('\n')[0])
  return sentences

In [39]:
train_source = get_sentences_from_file('train-source.txt')
train_target = get_sentences_from_file('train-target.txt')

In [36]:
test_source = get_sentences_from_file('test-source.txt')
test_target = get_sentences_from_file('test-target.txt')

In [29]:
len(train_source) == len(train_target)

True

In [28]:
train_source, val_source = train_source[:-5000], train_source[-5000:]
train_target, val_target = train_target[:-5000], train_target[-5000:]

In [2]:
def emission_prob(sentences):
  emission_probs = dict()
  for sentence in sentences:
    for word in sentence:
      if word not in emission_probs: emission_probs[word] = 1
      else: emission_probs[word] += 1
  total_sum = sum(list(emission_probs.values()))
  for word in emission_probs.keys():
    emission_probs[word] /= total_sum
  return emission_probs 

In [40]:
train_target_emission_probs = emission_prob(train_target)

In [41]:
len(train_target_emission_probs)

29116

In [18]:
def source_to_target_words_map(source, target):
  transmission_counts, transmission_probs = dict(), dict()
  for i in range(len(source)):
    for j in range(min(len(source[i]), len(target[i]))):
      if source[i][j] not in transmission_counts:
        transmission_counts[source[i][j]] = {target[i][j] : 1}
      elif target[i][j] not in transmission_counts[source[i][j]]:
        transmission_counts[source[i][j]][target[i][j]] = 1
      else:
        transmission_counts[source[i][j]][target[i][j]] += 1
  
  

  for key, data_dict in transmission_counts.items():
      transmission_probs[key] = {}
      dict_sum = sum(list(data_dict.values()))

      for new_key, value in data_dict.items():
          transmission_probs[key][new_key] = (transmission_counts[key][new_key] / dict_sum)

    
  return transmission_probs

In [42]:
transmission_probabilities = source_to_target_words_map(train_source, train_target)

In [34]:
predicted_sentence = predict(val_source, transmission_probabilities)

In [35]:
from nltk.translate.bleu_score import corpus_bleu
new_targ = []

for row in val_target:
    new_targ.append([row])

print(corpus_bleu(new_targ, predicted_sentence))

0.6882449272360074


In [43]:
predicted_test_sentences = predict(test_source, transmission_probabilities)

In [44]:
new_targ = []

for row in test_target:
    new_targ.append([row])

print(corpus_bleu(new_targ, predicted_test_sentences))

0.7421058136552283
